In [ ]:
# Hi,
# What do you want to do for the final project? Any ideas? I'm happy to connect..
# https://www.linkedin.com/in/viviankaun/


In [ ]:
!pip install -U langchain-community pypdf langchain-openai
from openai import OpenAI
from google.colab import userdata
open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/

In [ ]:

!wget "https://www.accenture.com/content/dam/accenture/final/accenture-com/document-2/Accenture-Life-Trends-2024-Full-Report.pdf"
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('Accenture-Life-Trends-2024-Full-Report.pdf')
pages = loader.load_and_split()


--2024-07-26 06:00:08--  https://www.accenture.com/content/dam/accenture/final/accenture-com/document-2/Accenture-Life-Trends-2024-Full-Report.pdf
Resolving www.accenture.com (www.accenture.com)... 108.138.64.112, 108.138.64.78, 108.138.64.84, ...
Connecting to www.accenture.com (www.accenture.com)|108.138.64.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30934749 (30M) [application/pdf]
Saving to: ‘Accenture-Life-Trends-2024-Full-Report.pdf’

Accenture-Life-Tren 100%[===================>]  29.50M  47.7MB/s    in 0.6s    

2024-07-26 06:00:09 (47.7 MB/s) - ‘Accenture-Life-Trends-2024-Full-Report.pdf’ saved [30934749/30934749]



#langchain.chains.combine_documents.stuff

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

# Define prompt
prompt_template = """Write a concise summary in a maximum of 10 bullet points of the following text enclosed within three backticks.:
```{text}```
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", api_key=open_ai_key)
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

res = stuff_chain.invoke(pages[1:56])
res["output_text"]


'- Accenture Song creates annual trends to explore the interplay between people and their behaviors, focusing on customer-obsession as a growth strategy.\n- Trends examine shifts in customer behavior and aim to help businesses stay relevant to customers.\n- The Accenture Life Trends 2024 report delves into various layers that influence how people live their lives.\n- It highlights the fragility in relationships between people and societal influences, leading to a state of flux in society.\n- The report discusses Trend 1, "Where\'s the love?", which focuses on the impact of cuts on customer obsession and customer experiences.\n- It also covers Trend 2, "The great interface shift", which explores how generative AI is upgrading the internet experience.\n- Trend 3, "Meh-diocrity", discusses the challenges of creativity being hindered'

# from langchain.chains.combine_documents.map_reduce

In [ ]:
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain

# Define MapReduceDocumentsChain prompts
map_prompt_template = """write a concise summary in a maximum of 10 bullet points of the following text enclosed within three backticks.
```{text}```
CONCISE SUMMARY:"""
map_prompt = PromptTemplate.from_template(map_prompt_template)

reduce_prompt_template = """Combine the following summaries into a single concise summary:
```{text}```
FINAL SUMMARY:"""
reduce_prompt = PromptTemplate.from_template(reduce_prompt_template)

# Define LLM for MapReduceDocumentsChain
map_llm_chain = LLMChain(llm=llm, prompt=map_prompt)
reduce_llm_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Define LLM for MapReduceDocumentsChain
map_llm_chain = LLMChain(llm=llm, prompt=map_prompt)
reduce_llm_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Define a StuffDocumentsChain to use as the reduce_documents_chain for MapReduce
reduce_stuff_chain = StuffDocumentsChain(llm_chain=reduce_llm_chain, document_variable_name="text")

# Define MapReduceDocumentsChain
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_llm_chain,
    reduce_documents_chain=reduce_stuff_chain,
    document_variable_name="text"
)

# Invoke MapReduceDocumentsChain
res_map_reduce = map_reduce_chain.invoke(pages[1:56])
res_map_reduce["output_text"]

'Accenture Life Trends 2024 explores the evolving relationship between people and technology, challenging traditional norms and milestones. The trends highlight the need for brands to prioritize customer relationships, adapt to changing societal values, and embrace sustainability. The text discusses themes such as mediocrity, the decade of deconstruction, the great interface shift, and the importance of love in technology and brand interactions. Error 429: Human request limit reached indicates the overwhelming pace of change and the need for more efficient communication.'

#Findings: Map-Rduce spent 1 mins to response, but StuffDocumentsChain spent 4 sec to resposne.


In [ ]:
import sqlite3
# Step 1: Connect to the database or create it if it doesn't exist
conn = sqlite3.connect('./sql_lite_database7.db')

#Step 2: Create a cursor
cursor = conn.cursor()
query =  """
                        CREATE TABLE IF NOT EXISTS "ORDERS"
                        (
                            "ORD_NUM" NUMBER(6,0) NOT NULL PRIMARY KEY,
                            "ORD_AMOUNT" NUMBER(12,2) NOT NULL,
                            "ADVANCE_AMOUNT" NUMBER(12,2) NOT NULL,
                            "ORD_DATE" DATE NOT NULL,
                            "CUST_CODE" VARCHAR2(6) NOT NULL  ,
                            "AGENT_CODE" CHAR(6) NOT NULL  ,
                            "ORD_DESCRIPTION" VARCHAR2(60) NOT NULL
                        ); """
cursor.execute(query)
query =  """
INSERT INTO ORDERS VALUES('200100', '1000.00', '600.00', '2008-08-01', 'C00013', 'A003', 'SOD');
INSERT INTO ORDERS VALUES('200117', '800.00', '200.00', '2008-10-20', 'C00014', 'A001', 'SOD');
INSERT INTO ORDERS VALUES('200123', '500.00', '100.00', '2008-09-16', 'C00022', 'A002', 'SOD');
INSERT INTO ORDERS VALUES('200124', '500.00', '100.00', '2008-09-16', 'C00022', 'A004', 'SOD');
"""
for row in query.splitlines():
    try:
        cursor.execute(row)
    except:
        print(f"An error occurred")
        print(row)
#Step 3: Close the cursor and connection
cursor.close()
conn.commit()
conn.close()

#Utilized SQL Agent to answer user questions by retrieving relevant data from the database. plus few-shot

In [ ]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import OpenAI
from langchain.agents import create_sql_agent

# define the database we want to use for our test
db = SQLDatabase.from_uri('sqlite:///sql_lite_database7.db')

# choose llm model, in this case the default OpenAI model
llm = OpenAI(
            temperature=0,
            verbose=True,
            openai_api_key=open_ai_key
            )
# setup agent
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors= True,
)



In [ ]:
%ls -l

total 30228
-rw-r--r-- 1 root root 30934749 Oct 18  2023 Accenture-Life-Trends-2024-Full-Report.pdf
drwxr-xr-x 1 root root     4096 Jul 24 13:22 sample_data/
-rw-r--r-- 1 root root    12288 Jul 26 06:01 sql_lite_database7.db


In [ ]:
question = input("Please enter your question: ")
agent_executor.invoke(question)

Please enter your question: How many orders do we have ?


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:
ORDERS I should query the ORDERS table to get the number of orders.
Action: sql_db_schema
Action Input: ORDERS
CREATE TABLE "ORDERS" (
	"ORD_NUM" NUMERIC(6, 0) NOT NULL, 
	"ORD_AMOUNT" NUMERIC(12, 2) NOT NULL, 
	"ADVANCE_AMOUNT" NUMERIC(12, 2) NOT NULL, 
	"ORD_DATE" DATE NOT NULL, 
	"CUST_CODE" TEXT(6) NOT NULL, 
	"AGENT_CODE" CHAR(6) NOT NULL, 
	"ORD_DESCRIPTION" TEXT(60) NOT NULL, 
	PRIMARY KEY ("ORD_NUM")
)

/*
3 rows from ORDERS table:
ORD_NUM	ORD_AMOUNT	ADVANCE_AMOUNT	ORD_DATE	CUST_CODE	AGENT_CODE	ORD_DESCRIPTION
200100	1000.00	600.00	2008-08-01	C00013	A003	SOD
200117	800.00	200.00	2008-10-20	C00014	A001	SOD
200123	500.00	100.00	2008-09-16	C00022	A002	SOD
*/ I should query the ORDERS table and count the number of rows.
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM ORDERS[(4,)] I now know the final answer
Final Answer: 4

> Finishe

{'input': 'How many orders do we have ?', 'output': '4'}

In [ ]:

few_shot_examples = """
Q: How many customers do we have in our database?
A: We currently have 4 customers in our database.

Q: What is the total revenue generated the year 2008)?
A: The total revenue generated this year is $2800,000, ORD_DATE>='2008-01-01' AND ORD_DATE<='2008-12-31'

Q: Please provide a detailed response to the following question: {question}
"""

# Define the user's question
question = input("Please enter your question: ")
few_shot_prompt = few_shot_examples.format(question=question)
agent_executor.invoke(few_shot_prompt)

Please enter your question: What is the highest revenue in a day?


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ORDERS I should query the ORDERS table to see what columns I can use to find the highest revenue in a day.
Action: sql_db_schema
Action Input: ORDERS
CREATE TABLE "ORDERS" (
	"ORD_NUM" NUMERIC(6, 0) NOT NULL, 
	"ORD_AMOUNT" NUMERIC(12, 2) NOT NULL, 
	"ADVANCE_AMOUNT" NUMERIC(12, 2) NOT NULL, 
	"ORD_DATE" DATE NOT NULL, 
	"CUST_CODE" TEXT(6) NOT NULL, 
	"AGENT_CODE" CHAR(6) NOT NULL, 
	"ORD_DESCRIPTION" TEXT(60) NOT NULL, 
	PRIMARY KEY ("ORD_NUM")
)

/*
3 rows from ORDERS table:
ORD_NUM	ORD_AMOUNT	ADVANCE_AMOUNT	ORD_DATE	CUST_CODE	AGENT_CODE	ORD_DESCRIPTION
200100	1000.00	600.00	2008-08-01	C00013	A003	SOD
200117	800.00	200.00	2008-10-20	C00014	A001	SOD
200123	500.00	100.00	2008-09-16	C00022	A002	SOD
*/ I should query the ORDERS table and use the MAX function to find the highest revenue in a day.
Action: sql_db_query
Action Input: SELECT 

{'input': "\nQ: How many customers do we have in our database?\nA: We currently have 4 customers in our database.\n\nQ: What is the total revenue generated the year 2008)?\nA: The total revenue generated this year is $2800,000, ORD_DATE>='2008-01-01' AND ORD_DATE<='2008-12-31'\n\nQ: Please provide a detailed response to the following question: What is the highest revenue in a day?\n",
 'output': 'The highest revenue in a day is $1000.'}

In [ ]:
question = input("Please enter your question: ")
agent_executor.invoke(question)

Please enter your question:  What is highest revenue we earned in single day?


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ORDERSI should query the ORDERS table to find the highest revenue earned in a single day.
Action: sql_db_schema
Action Input: ORDERS
CREATE TABLE "ORDERS" (
	"ORD_NUM" NUMERIC(6, 0) NOT NULL, 
	"ORD_AMOUNT" NUMERIC(12, 2) NOT NULL, 
	"ADVANCE_AMOUNT" NUMERIC(12, 2) NOT NULL, 
	"ORD_DATE" DATE NOT NULL, 
	"CUST_CODE" TEXT(6) NOT NULL, 
	"AGENT_CODE" CHAR(6) NOT NULL, 
	"ORD_DESCRIPTION" TEXT(60) NOT NULL, 
	PRIMARY KEY ("ORD_NUM")
)

/*
3 rows from ORDERS table:
ORD_NUM	ORD_AMOUNT	ADVANCE_AMOUNT	ORD_DATE	CUST_CODE	AGENT_CODE	ORD_DESCRIPTION
200100	1000.00	600.00	2008-08-01	C00013	A003	SOD
200117	800.00	200.00	2008-10-20	C00014	A001	SOD
200123	500.00	100.00	2008-09-16	C00022	A002	SOD
*/ I should query the ORDERS table for the highest revenue earned in a single day by ordering the results by ORD_AMOUNT in descending order and l

{'input': ' What is highest revenue we earned in single day?',
 'output': 'The highest revenue earned in a single day is $1000.'}